# Along track binning
### For mapping methods that take as input gridded observations instead of raw along tracks, we provide this binning script that interpolates simulated along tracks observations on a daily grid whose spatial resolution is left to the user's choice (1/8° or 1/20°).

In [ ]:
import glob
import xarray as xr
import pyinterp
import numpy as np
import pyinterp
import os

## 1. Input

### 1.1 Choice of the target grid : 1/8° of 1/20°

In [ ]:
path_to_grid = '/material_data_challenge/NATL60-CJM165/reg_1_8_daily.zarr' # or '/material_data_challenge/NATL60-CJM165/reg_1_20_daily.zarr'

In [ ]:
ds_ssh_maps = xr.open_zarr(path_to_grid)
ds_ssh_maps

In [ ]:
lon_is_circle = False

### 1.2 Choice of the constellation to be binned

In [ ]:
alongtrack_folder = '/material_data_challenge/NATL60-CJM165/alongtrack'

Choose the constellation :

In [ ]:
list_mission_of_interest = ['alg', 'c2', 'h2ag', 'h2b', 'j3', 's3a', 's3b'] 

### 1.3 Choice of the period of interest

In [ ]:
vtime = np.arange(np.datetime64('2012-12-01'), np.datetime64('2013-01-01'))
vtime

## 2. Output 

In [ ]:
output_dir = '../alongtrack_binning_1_8'
os.system(f'mkdir -p {output_dir}')

## 3. Binning

In [ ]:
def alongtrack_binning(date):
    
    date_of_interest = np.datetime_as_string(date, unit='D')
    
    # Select files for one particular date of interest
    list_of_file = []
    for cmission in list_mission_of_interest:
        date_of_interest2 = date_of_interest.replace('-', '')
        list_of_file.append(glob.glob(f'{alongtrack_folder}/{cmission}/*/dt_global_{cmission}_phy_l3_{date_of_interest2}_*.nc'))
    list_of_file = [item for sublist in list_of_file for item in sublist]
    print(list_of_file)
    
    # Read alongtrack of interest
    ds_alongtrack = xr.open_mfdataset(list_of_file, combine='nested', concat_dim='time')
    # change alongtrack lon between -180__180
    ds_alongtrack['longitude'] = ds_alongtrack['longitude'].where(ds_alongtrack['longitude'] <=180, ds_alongtrack['longitude'] - 360.)
    
    # create binning grid
    lon_out = ds_ssh_maps.lon
    lat_out = ds_ssh_maps.lat
    binning = pyinterp.Binning2D(pyinterp.Axis(lon_out, is_circle=lon_is_circle), 
                                 pyinterp.Axis(lat_out))
    binning.clear()
    binning.push(ds_alongtrack['longitude'], ds_alongtrack['latitude'], ds_alongtrack['ssh_norm'], True)
    mean_sla = binning.variable('mean').T
    
    ds_out = xr.Dataset({'ssh_norm' : (('time', 'lat', 'lon'), np.expand_dims(mean_sla, axis=0)) },
                               coords={'time': [date],
                                       'lon': lon_out, 
                                       'lat': lat_out, 
                                       })
    
    ds_out.to_netcdf(f'{output_dir}/alongtrack_binning_{date_of_interest2}.nc')
    
    

    

In [ ]:
for time in vtime:
    alongtrack_binning(time)